In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime as dt

from db_tools import read_csv_covariance, read_csv_profiles

%matplotlib inline

## COSMO7

- Interpolate to radiosonde launch time
- Remove bias wrt to radiosonde
- Save bias-corrected values and covariance matrix of errors

In [ ]:
def interpolate(target, dfs, hourbounds=(0, 6)):
    lower = min(min(df.index) for df in dfs)
    upper = max(max(df.index) for df in dfs)
    idx = pd.Index([
            d for d in target.index
                    if lower <= d <= upper
                    and hourbounds[0] <= d.hour <= hourbounds[1]
            ], name="valid")
    out = pd.DataFrame(index=idx.union_many(df.index for df in dfs), columns=target.columns, dtype=float)
    for df in dfs:
        out.ix[df.index] = df
    return idx, out.interpolate(method="time", axis=0).ix[idx]

In [ ]:
def cosmo_prior(raso, dfl, dfh):
    idx, itp = interpolate(raso, [dfl, dfh])
    diff = raso.ix[idx] - itp
    return itp + diff.mean(), diff.cov()

### Temperature Prior

In [ ]:
raso = read_csv_profiles("../data/unified/T_raso.csv")
cosmo00 = read_csv_profiles("../data/unified/T_cosmo7+00.csv")
cosmo06 = read_csv_profiles("../data/unified/T_cosmo7+06.csv")
cosmo24 = read_csv_profiles("../data/unified/T_cosmo7+24.csv")
cosmo30 = read_csv_profiles("../data/unified/T_cosmo7+30.csv")

means0006, cov0006 = cosmo_prior(raso, cosmo00, cosmo06)
means2430, cov2430 = cosmo_prior(raso, cosmo24, cosmo30)

means0006.to_csv("../data/unified/priors/T_cosmo7+00+06_mean.csv")
means2430.to_csv("../data/unified/priors/T_cosmo7+24+30_mean.csv")
cov0006.to_csv("../data/unified/priors/T_cosmo7+00+06_cov.csv")
cov2430.to_csv("../data/unified/priors/T_cosmo7+24+30_cov.csv")

### Humidity Prior

In [ ]:
raso = read_csv_profiles("../data/unified/qvap_raso.csv")
cosmo00 = read_csv_profiles("../data/unified/qvap_cosmo7+00.csv")
cosmo06 = read_csv_profiles("../data/unified/qvap_cosmo7+06.csv")
cosmo24 = read_csv_profiles("../data/unified/qvap_cosmo7+24.csv")
cosmo30 = read_csv_profiles("../data/unified/qvap_cosmo7+30.csv")

raso += read_csv_profiles("../data/unified/qliq_raso.csv")
cosmo00 += read_csv_profiles("../data/unified/qliq_cosmo7+00.csv")
cosmo06 += read_csv_profiles("../data/unified/qliq_cosmo7+06.csv")
cosmo24 += read_csv_profiles("../data/unified/qliq_cosmo7+24.csv")
cosmo30 += read_csv_profiles("../data/unified/qliq_cosmo7+30.csv")

means0006, cov0006 = cosmo_prior(np.log(raso), np.log(cosmo00), np.log(cosmo06))
means2430, cov2430 = cosmo_prior(np.log(raso), np.log(cosmo24), np.log(cosmo30))

means0006.to_csv("../data/unified/priors/lnq_cosmo7+00+06_mean.csv")
means2430.to_csv("../data/unified/priors/lnq_cosmo7+24+30_mean.csv")
cov0006.to_csv("../data/unified/priors/lnq_cosmo7+00+06_cov.csv")
cov2430.to_csv("../data/unified/priors/lnq_cosmo7+24+30_cov.csv")